In [1]:
%pwd #print the current working directory

'c:\\Users\\oanuf\\Google Drive\\Programming\\GitHub\\Alpaca_trading'

In [1]:
import pandas as pd
import datetime as dt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Importing Alpaca SDK
from Alpaca_config import *

import fmpsdk

import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame
from alpaca_trade_api.rest import TimeFrameUnit
from pytz import timezone

alpaca = tradeapi.REST(API_KEY_PAPER, API_SECRET_PAPER, API_BASE_URL_PAPER, 'v2')

### Earnings Calls Trading Check

Earning calls are not legally mandated, so a company doesn’t actually have to have one. Public companies are required to release the details of their financial performance, but their earnings don’t have to be amongst the details released. 

Some publicly traded companies don’t even have earnings calls.

Earning calls tend to take place on a quarterly basis. Oftentimes, a company will announce the date and time of its call several weeks out.

How long are Earnings Calls: Expect the call to last between 45 and 60 minutes.

In [3]:
dateparse = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
hist_earn_calls=pd.read_excel('D:\\Data\\Other_data\\all_5Y_earn_calls.xlsx',
                                usecols=['RIC','Ticker', 'ISIN', 'Event_Type', 'Event_date_GMT'],
                                parse_dates=['Event_date_GMT'], date_parser=dateparse)
hist_earn_calls.dtypes
hist_earn_calls

RIC                       object
Ticker                    object
ISIN                      object
Event_Type                object
Event_date_GMT    datetime64[ns]
dtype: object

,RIC,Ticker,ISIN,Event_Type,Event_date_GMT
0,SNI.OQ^C18,SNI,US8110651010,EarningsReleases,2017-02-21 13:30:00
1,SNI.OQ^C18,SNI,US8110651010,EarningsCallsAndPresentations,2017-02-21 15:00:00
2,SNI.OQ^C18,SNI,US8110651010,EarningsReleases,2017-05-04 12:30:00
3,SNI.OQ^C18,SNI,US8110651010,EarningsCallsAndPresentations,2017-05-04 14:00:00
4,SNI.OQ^C18,SNI,US8110651010,EarningsReleases,2017-07-31 12:30:00
...,...,...,...,...,...
16886,FDX.N,FDX,US31428X1063,EarningsCallsAndPresentations,2021-06-24 21:00:00
16887,FDX.N,FDX,US31428X1063,EarningsReleases,2021-09-21 20:04:00
16888,FDX.N,FDX,US31428X1063,EarningsCallsAndPresentations,2021-09-21 21:30:00
16889,FDX.N,FDX,US31428X1063,EarningsReleases,2021-12-16 21:02:00


In [17]:

fig1 = px.bar(hist_earn_calls[hist_earn_calls['Event_Type']=='EarningsReleases']['RIC'].groupby([hist_earn_calls['Event_date_GMT'].dt.hour]).count(),
                title="Number of earning releases by hour")
fig1.update_layout(showlegend=False,title_x=0.5).update_xaxes(title_text='Hour of the day (GMT)',nticks=24).update_yaxes(title_text='Num of Earning Releases')
fig2 = px.bar(hist_earn_calls[hist_earn_calls['Event_Type']=='EarningsReleases']['RIC'].groupby([hist_earn_calls['Event_date_GMT'].dt.date]).count(),
            title="Number of earning releases by date").update_traces(marker_color='red')
fig2.update_layout(showlegend=False,title_x=0.5)
fig3 = px.bar(hist_earn_calls[hist_earn_calls['Event_Type']=='EarningsReleases']['RIC'].groupby([hist_earn_calls['Event_date_GMT'].dt.month]).count(),
            title="Number of earning releases by month").update_traces(marker_color='green')
fig3.update_layout(showlegend=False,title_x=0.5).update_xaxes(title_text='Month',nticks=12).update_yaxes(title_text='Num of Earning Releases')



In [ ]:
end = hist_earn_calls['Event_date_GMT'].dt.date.max().strftime("%Y-%m-%d") # latest date in the db
start = hist_earn_calls['Event_date_GMT'].dt.date.min().strftime("%Y-%m-%d") # latest date in the db
symbols = list(set(hist_earn_calls['Ticker']))